# Simple discretization methods

In [47]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import normalize
from sklearn import preprocessing

import plotly.express as px
import plotly.graph_objects as go

# Generation example dataframe

In [48]:
sample_size = 1000

np.random.seed(432409)

example_data = pd.DataFrame(
    {
        "var" : np.random.uniform(-200, 200, size = sample_size)
    }
)

example_data["class"] = list(map(
    lambda p: np.random.choice([0,1], p=[1-p, p]),
    (example_data["var"] - example_data['var'].min())/\
    (example_data["var"].max() - example_data["var"].min())
))

In [61]:
fig = go.Figure()

colors = ["blue", 'red']

for cl in example_data["class"].unique():
    
    fig.add_trace(
        go.Histogram(
            x = example_data.query("class == '" + str(cl) + "'")["var"],
            marker_color = colors[cl],
            opacity = 0.75
        )
    )

#fig.update_layout(barmode='overlay')
fig.update_traces(opacity=0.75)

fig.show()
#px.histogram(example_data, x = "var", color = 'class', nbins = 40)

SyntaxError: Python keyword not valid identifier in numexpr query (<unknown>, line 1)

In [62]:
example_data.query('class == 1')

SyntaxError: Python keyword not valid identifier in numexpr query (<unknown>, line 1)

In [5]:


x_array = np.array([2,3,5,6,7,4,8,7,6])
normalized_arr = preprocessing.normalize([x_array])
print(normalized_arr)

[[0.11785113 0.1767767  0.29462783 0.35355339 0.41247896 0.23570226
  0.47140452 0.41247896 0.35355339]]
